## NFA, accepting all strings that end in 01

The template is from Hopcroft et. al.: Introduction to Automata Theory, Languages and Computation, chapter 2.3.1, example 2.6. We examine Julia's multiple dispatch feature on application to state machines, in this case to a NFA.

Hopcroft's example automaton has three single states $q_0, q_1, q_2$. Since an NFA is nondeterministic, its transition functions return sets of states, e.g. $\{q_0\}, \{q_0, q_1\} \dotsc$.

1. We define a fallback transition function $\delta_0$, which returns an empty set $\emptyset$.
2. Then we need three elementary transition functions: $\delta_1(q_0, 0) = \{q_0, q_1\}$, $\delta_2(q_0, 1) = \{q_0\}$ and $\delta_3(q_1, 1) = \{q_2\}$. 
3. Next we have a dispatching transition function $\bar{\delta}$, which takes a set of states and an input $\sigma$ and – after employing the single transition functions on each state – delivers another set of states. 
4. Finally we have the extended transition function $\hat{\delta}$, taking a word $w$ as input and recursively employing the dispatching transition function on each letter.

NB: sets here are implemented as Julia tuples.

In [1]:
using Base.Iterators
@enum State q0 q1 q2

δ(::Any, ::Any)   = () 

δ(::Val{q0},::Val{0}) = (q0,q1)
δ(::Val{q0},::Val{1}) = (q0,)
δ(::Val{q1},::Val{1}) = (q2,)

δ(states::Tuple, σ::Int64) = Tuple(union(flatten( [δ(Val(q),Val(σ)) for q ∈ states] )))

δ(states::Tuple,w::Array{Int64}) = length(w) == 1 ? δ(states,w[1]) : δ(δ(states,w[1]),w[2:end]) 

δ (generic function with 6 methods)

All those transition functions are named `δ`, and Julia's multiple dispatch chooses between them at runtime according to the parameters: states and inputs.

The following is Hopcroft's example for a string of $00101$.

In [2]:
a = δ(Val(q0),Val(0)); println("a: ", a)
b = δ(a,0); println("b: ", b)
c = δ(b,1); println("c: ", c)
d = δ(c,0); println("d: ", d)
e = δ(d,1); println("e: ", e)

a: (q0, q1)
b: (q0, q1)
c: (q0, q2)
d: (q0, q1)
e: (q0, q2)


In [3]:
δ((q0,),[0,0,1,0,1])

(q0, q2)

We introduce a `check`-function to determine if `w` is a word accepted by the NFA. Since $q_2$ is the accepting state, the acceptance criterion is $q_2 \in \{\text{set of states}\}$.

In [4]:
check(w::Array{Int64}) = q2 ∈ δ((q0,), w)
check(w::Int64) = check(reverse(digits(w, base=2)))
check(w::String) = check([Int64(i)-48 for i ∈ w])

check (generic function with 3 methods)

In [5]:
check([0,0,1,0,1])

true

In [6]:
check("00101")

true

In [7]:
check(1025)

true

In [8]:
check(1026)

false

Note that alternatively the extended transition function $\hat{\delta}$ could be implemented as a "flat" version with a `for` loop.

In [9]:
# extended transition function, flat version
function δ(states::Tuple,w::Array{Int64})
    for σ ∈ w
        states = δ(states,σ)
    end
    states
end

δ (generic function with 6 methods)

In [10]:
check(1025)

true

The whole approach is not yet robust against arbitrary strings since the automaton delivers an empty set $\emptyset$ if it meets an input other than the allowed $\{0, 1\}$. It then has a undefined state.

In [11]:
check("aköjbköjbh01")

ArgumentError: ArgumentError: Union{} does not have elements

## Improving and simplifying

For simplicity we can delete the transition function $\delta_2(q_0, 1) = \{q_0\}$ and redefine our fallback transition function as $\delta_0 = \{q_0\}$.

In [12]:
δ₂ = @which δ(Val(q0),Val(1))
Base.delete_method(δ₂)

δ(::Any,::Any) = (q0,)

δ (generic function with 5 methods)

Our NFA then has 5 methods (3 for switching states) and returns to its ground state $\{q0\}$ for every unexpected input. It runs on arbitrary strings.

In [13]:
check("aköjbköjbh01")

true

In [14]:
check("aköjbköjbh011")

false

For completeness I give here the definition of the improved NFA $A=(Q,\Sigma,\delta,q_0,F)$:

1. $\delta_0 = \{q_0\}$, fallback transition function
2. $\delta_1(q_0, 0) = \{q_0, q_1\}$ 
3. $\delta_2(q_1, 1) = \{q_2\}$
4. $\bar{\delta}(Q_x \in Q, \sigma) = Q_y \in Q$, dispatching transition function working on a set of states.
5. $\hat{\delta}(Q_x \in Q, w) = Q_y \in Q$, extended transition function working on a word.

… together with their implementation in Julia:

In [15]:
δ(::Any,::Any) = (q0,)
δ(::Val{q0},::Val{0}) = (q0,q1)
δ(::Val{q1},::Val{1}) = (q2,)
δ(states::Tuple, σ::Int64) = Tuple(union(flatten( [δ(Val(q),Val(σ)) for q ∈ states] )))
δ(states::Tuple,w::Array{Int64}) = length(w) == 1 ? δ(states,w[1]) : δ(δ(states,w[1]),w[2:end]) 

δ (generic function with 5 methods)